In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.1 MB/s eta 0:00:00


In [ ]:
import requests
import json
import pandas as pd
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder

# Download the dataset
url = "https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/Rhetorical_Role_Benchmark/Data/train.json"
response = requests.get(url)
json_data = response.json()

# Convert JSON to DataFrame
def json_to_dataframe(json_data):
    data = []
    for document in json_data:
        doc_id = document.get("id")
        for annotation in document.get("annotations", []):
            for result in annotation.get("result", []):
                segment = {
                    'doc_id': doc_id,
                    'text': result['value'].get('text'),
                    'label': result['value'].get('labels', [None])[0]  # Get the first label if available
                }
                data.append(segment)
    return pd.DataFrame(data)

df = json_to_dataframe(json_data)

# Encode the labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])


def add_positional_info(df):
    # Sort the dataframe by document ID to group sentences from the same document
    df = df.sort_values(by=['doc_id'])

    # Add absolute position, normalized position, and k-quantile position
    df['absolute_pos'] = df.groupby('doc_id').cumcount() + 1  # Starts from 1 for each document
    df['doc_length'] = df.groupby('doc_id')['text'].transform('count')
    df['normalized_pos'] = df['absolute_pos'] / df['doc_length']
    df['k_quantile_pos'] = pd.qcut(df['absolute_pos'], q=4, labels=False)  # 4-quantile position

    return df

# Apply positional information to the dataframe
df = add_positional_info(df)

# Drop the 'doc_length' column as it's not needed for tokenization
df = df.drop(columns=['doc_length'])

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)


# Split the dataset into train, test, and eval sets
train_test_split = dataset.train_test_split(test_size=0.2, seed=42)
test_eval_split = train_test_split['test'].train_test_split(test_size=0.5, seed=42)

# Combine into a DatasetDict
split_dataset = {
    'train': train_test_split['train'],
    'test': test_eval_split['test'],
    'eval': test_eval_split['train']
}

# Verify the split
print("Train size:", len(split_dataset['train']))
print("Test size:", len(split_dataset['test']))
print("Eval size:", len(split_dataset['eval']))


Train size: 23188
Test size: 2899
Eval size: 2899


In [ ]:
dataset.shape

(28986, 7)

In [ ]:
# Add sentence position embeddings to the dataset
def add_position_embeddings(df):
    df['absolute_pos'] = df.groupby('doc_id').cumcount() + 1
    df['normalized_pos'] = df['absolute_pos'] / df.groupby('doc_id')['absolute_pos'].transform('max')

    def quantile_position(group):
        group['quantile_pos'] = pd.qcut(group['absolute_pos'], q=4, labels=False) + 1  # k=4 quantiles
        return group

    df = df.groupby('doc_id').apply(quantile_position)
    return df

df = add_position_embeddings(df)


<ipython-input-3-56e39bd65582>:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('doc_id').apply(quantile_position)


In [ ]:
df.head(28986)

doc_id                                               text  label  \
doc_id                                                                          
1711   5130    1711  It would be even more anomalous to have an app...      0   
       5168    1711  As the Punjab Courts Act does not contemplate ...      0   
       5169    1711  The Court contemplated is the Court of the Add...      0   
       5170    1711  We hold therefore that the Court of the Additi...      0   
       5171    1711  \n Now, as we have seen, when the original Cou...      0   
...             ...                                                ...    ...   
13658  9485   13658  The accused pleaded not guilty and claims to b...      0   
       9486   13658  Hence, case has been posted for recording of e...      0   
       9487   13658  \n 7. In order to substantiate the case made o...      0   
       9489   13658                           No defence evidence led.      5   
       9492   13658            Perused the materials placed on record.      0   

             absolute_pos  normalized_pos  k_quantile_pos  quantile_pos  
doc_id                                                                   
1711   5130             1        0.005319               0             1  
       5168             2        0.010638               0             1  
       5169             3        0.015957               0             1  
       5170             4        0.021277               0             1  
       5171             5        0.026596               0             1  
...                   ...             ...             ...           ...  
13658  9485            69        0.945205               2             4  
       9486            70        0.958904               2             4  
       9487            71        0.972603               2             4  
       9489            72        0.986301               2             4  
       9492            73        1.000000               2             4  

[28986 rows x 7 columns]

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import torch.nn as nn
from transformers import BertModel

class BERTWithPositionalEmbeddings(nn.Module):
    def __init__(self, model_name, num_labels):
        super(BERTWithPositionalEmbeddings, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None, absolute_pos=None, normalized_pos=None, k_quantile_pos=None):
        # Pass through BERT
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        sequence_output = outputs[0]  # Last hidden state

        # Here you can incorporate positional information (absolute_pos, normalized_pos, k_quantile_pos) if necessary

        # Pass through classifier
        pooled_output = outputs[1]  # CLS token's output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        # If labels are provided, calculate loss
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.classifier.out_features), labels.view(-1))

        # Return the loss and logits
        if loss is not None:
            return loss, logits
        else:
            return logits

# Load the tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Initialize the model
model = BERTWithPositionalEmbeddings(model_name=model_name, num_labels=len(label_encoder.classes_))



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
model

BERTWithPositionalEmbeddings(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-

In [ ]:

def preprocess_function(examples):
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True)
    tokenized_inputs['absolute_pos'] = examples['absolute_pos']
    return tokenized_inputs

# Tokenize the datasets
tokenized_datasets = {}
for split in ['train', 'test', 'eval']:
    tokenized_datasets[split] = split_dataset[split].map(preprocess_function, batched=True)



Map:   0%|          | 0/23188 [00:00<?, ? examples/s]

Map:   0%|          | 0/2899 [00:00<?, ? examples/s]

Map:   0%|          | 0/2899 [00:00<?, ? examples/s]

In [ ]:
pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00


In [ ]:
from transformers import Trainer, TrainingArguments
import wandb

# Initialize wandb
#wandb.init(project="rhetorical-role-bert", entity="your_wandb_username")  # Replace with your WandB username and project

# Step 3: Setup Training Arguments
training_args = TrainingArguments(
    output_dir="./resultsLR_Pos_BERTagain",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    save_total_limit=1,  # Keep only the most recent model
    save_steps=500,  # Save checkpoint every 500 steps
    logging_dir="./logs",
    learning_rate=2e-5,#new add
    weight_decay=0.01,#new add
    warmup_steps=100,#new add
    report_to="wandb",  # Report to Weights & Biases
    logging_steps=100,  # Log every 100 steps
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
)

# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['eval'],
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
0,1.155600,1.173184
1,0.964700,1.086361
2,0.852300,1.076046


TrainOutput(global_step=2172, training_loss=1.0977826425822839, metrics={'train_runtime': 6621.3699, 'train_samples_per_second': 10.506, 'train_steps_per_second': 0.328, 'total_flos': 0.0, 'train_loss': 1.0977826425822839, 'epoch': 2.996895481200414})

# Precision, Recall, F1 score, Accuracy (Evaluating Model)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import Trainer


# Define a compute_metrics function
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# If the model is already trained, just initialize the Trainer with the same model
trainer = Trainer(
    model=model,  # Use your already trained model
    #args=training_args,
    eval_dataset=tokenized_datasets['eval'],  # Use your evaluation dataset
    compute_metrics=compute_metrics  # Include the metrics function
)

# Evaluate the model
results = trainer.evaluate()

# Print the evaluation results
print(results)


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: engineersaloni159 (salonijnu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'eval_loss': 2.6256320476531982, 'eval_model_preparation_time': 0.0038, 'eval_accuracy': 0.09451535012073128, 'eval_precision': 0.42194022216665367, 'eval_recall': 0.09451535012073128, 'eval_f1': 0.04571065162758925, 'eval_runtime': 79.6703, 'eval_samples_per_second': 36.387, 'eval_steps_per_second': 4.556}


In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
# Make predictions
predictions = trainer.predict(tokenized_dataset)
predicted_labels = predictions.predictions.argmax(-1)

# Print predicted labels
print(predicted_labels)

KeyboardInterrupt: 

# Manually Evaluation of model

In [ ]:
!pip uninstall torch torchvision torchaudio -y

In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from datasets import load_dataset, Dataset
import pandas as pd
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("engineersaloni159/Positional_BERT_for_rhetorical_role_labeling")
model = AutoModelForSequenceClassification.from_pretrained("engineersaloni159/Positional_BERT_for_rhetorical_role_labeling")


config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
# Ensure model is in evaluation mode
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
#run this cell if model is directly called from  huggingface
from sklearn.preprocessing import LabelEncoder

# Assuming you already have this
label_encoder = LabelEncoder()

# Fit and transform the labels
df['label'] = label_encoder.fit_transform(df['label'])

# Print the mapping of labels to numbers
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)

NameError: name 'df' is not defined

In [ ]:
# Assuming label_encoder is the one you used for encoding labels earlier

# Example sentence
sentence =  "The High Court was then moved under s. 66 (2) of the Indian income-tax Act, 1922 and the High, Court heard the two applications together and directed the Tribunal to state a case on the following two questions which, in the opinion of the High Court, arose out of the Tribunal's orders.\n          (1) Whether the claim of loss in this case is governed by the provisions of S. 10(1) or 24(1) proviso read with s. 14(2)(c), or by the provisions of s. 42 ?\n (2) Whether on the facts of the case a loss of Rs. 22,981/- is allowable in computing the income of the assessee chargeable to the Excess Profits Tax ?"

# Tokenize the sentence
inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)

# Make predictions
with torch.no_grad():
    outputs = model(**inputs)

# The predictions are in the form of logits, you can convert them to probabilities
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1)

# Convert the predicted class number to the original label
predicted_label = label_encoder.inverse_transform(predicted_class.cpu().numpy())[0]

# Print the predicted label
print(f"Predicted label: {predicted_label}")


Predicted label: RLC


In [ ]:
print("Logits:", logits)


Logits: tensor([[-0.4151, -1.8779, -1.5828,  2.4168,  2.6604, -0.4153,  0.8213, -0.6055,
         -0.4555, -1.2325,  2.7832, -0.7909, -0.9192]])
